In [1]:
import numpy as np
import pandas as pd

In [2]:
complete = pd.read_excel('C:/Users/Owner/Documents/School/CMPT898/Field Data AGILE DataCollection 2016 Field Trials Nepal_03.07..2017.xlsx', sheetname='Measurements')
seed_weight = pd.read_excel('C:/Users/Owner/Documents/School/CMPT898/170621-1000sdwt (2016 set up).xlsx')

In [3]:
# Manually input climate data from the nepalese government website
# Starting from 2016-11-14 and going to 2016-12-21
GDD = np.array([11.75, 11.1, 11.45, 11.55, 12.3, 12.4, 12.5, 13.0, 13.45, 12.4, 12.0, 12.05, 11.8, 11.45, 11.5, 11.9, 12.05, 12.75, 12.15, 11.6, 11.5, 11.1, 10.35, 10.6, 10.05, 10.55, 
               10.7, 10.45, 9.95, 8.4, 8.6, 9.4, 9.2, 10.15, 9.35, 8.85, 9.0, 9.1]).cumsum()
precip = np.zeros(38)
precip[-4] = 4
precip = precip.cumsum()
daylength = np.array([12.78, 12.78, 12.77, 12.73, 12.72, 12.7, 12.68, 12.67, 12.65, 12.62, 12.62, 12.6, 12.58, 12.57, 12.57, 12.55, 12.53, 12.52, 12.52, 12.5, 12.48, 12.47, 12.47, 
                     12.45, 12.43, 12.43, 12.43, 12.42, 12.42, 12.42, 12.4, 12.4, 12.4, 12.4, 12.38, 12.38, 12.38, 12.38])

In [4]:
# Get an array of stack indexes from the plant dates, with the data sorted by entry
stack_idx = (complete.sort_values('entry')['Planting Date\n(date)'] > pd.datetime(2016, 11, 14)).values.astype(float) + (complete.sort_values('entry')['Planting Date\n(date)'] > pd.datetime(2016, 11, 15)).values.astype(float) + (complete.sort_values('entry')['Planting Date\n(date)'] > pd.datetime(2016, 11, 16)).values.astype(float)
# Use the index to create an array of values to remove from the cumulative sum if planted later than the 14th
GDD_adj = np.array(stack_idx)
GDD_adj[GDD_adj == 1] = GDD[0]
GDD_adj[GDD_adj == 2] = GDD[0] + GDD[1]
GDD_adj[GDD_adj == 3] = GDD[0] + GDD[1] + GDD[2]

# Precipitation requires no adjustment because the first three terms are 0

In [5]:
# Get the correct GDD. Everything is synced to 14th start date, so add extra days on the end and subtract days from the beginning using the adjustment array 
data = pd.DataFrame()
data = data.assign(Entry = complete.sort_values('entry')['entry'])
data = data.assign(Line = complete.sort_values('entry')['Name'])
# Convert spaces in line names to dashes 
data['Line'] = data['Line'].replace(' ', '-', regex = True)
data = data.assign(DTE = complete.sort_values('entry')['Days to Emergence\n(days)'])
data = data.assign(GDD = GDD[complete.sort_values('entry')['Days to Emergence\n(days)'].values + stack_idx.astype(int)] - GDD_adj)
data = data.assign(precip = precip[complete.sort_values('entry')['Days to Emergence\n(days)'].values + stack_idx.astype(int)])
data = data.assign(Seedwt = seed_weight['1000 Sdwt'][complete.sort_values('entry')['entry'].values.astype(int) - 1].values)
data = data.assign(Day = daylength[((complete.sort_values('entry')['Days to Emergence\n(days)'] + stack_idx).values / 2).astype(int)])
data = data.assign(Location = 'Nepal')
data = data.assign(Year = '2016')
data.columns = ['Entry', 'Line', 'DTE', 'GDD [C]', 'Precipitation [mm]', 'Seed 1000wt [g]', 'Day Length [hr]', 'Location', 'Year']

In [8]:
data.to_csv('Nepal Climate Data 2016.csv', index = False)